In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from collections import defaultdict, Counter

<Center><h2> Joey Logan <br><br> Data Exercise for Periop Healthcare Data Analyst</h2> </br>
<h2>  The Children’s Hospital of Philadelphia </h2></center>


In [2]:
patdf = pd.read_csv('patients.csv')  # Reading the patients csv

In [3]:
patdf.head() #exploring the data

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
0,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,1943-03-11,NaN,999-86-7250,S99939389,X3970685X,Mrs.,Allyn942,Kreiger457,NaN,Bartoletti50,M,asian,asian_indian,F,Muhlenberg Pennsylvania US,372 Marks Heights Unit 20,Middle Paxton,Pennsylvania,NaN
1,7f4ea9fb-f436-411e-ab34-e94750edfa93,1980-09-28,NaN,999-90-4314,S99920355,X28211313X,Mr.,Kieth891,King743,NaN,NaN,M,white,irish,M,Overfield Pennsylvania US,428 Strosin Fort Suite 49,Limerick,Pennsylvania,NaN
2,553b00b2-347c-48ec-90ee-f59924ff50eb,1973-02-22,NaN,999-78-9189,S99980820,X9988931X,Mrs.,Domenica436,Fadel536,NaN,Labadie908,M,white,scottish,F,Wilkins Pennsylvania US,519 Ziemann Trail,Washington,Pennsylvania,15301.0
3,50f799aa-740c-4da9-ab85-49d5cbf4046b,1955-03-30,NaN,999-49-5162,S99961309,X17464073X,Mr.,Kelly223,Turner526,NaN,NaN,M,white,irish,M,Manor Pennsylvania US,158 Dickens Corner Suite 2,Manheim,Pennsylvania,17545.0
4,bcef3b7a-0380-4b7e-b2b9-2caf07e41f0e,2009-05-24,NaN,999-74-7736,NaN,NaN,NaN,Zack583,Purdy2,NaN,NaN,NaN,white,german,M,Cranberry Pennsylvania US,1000 Vandervort Run,Washington,Pennsylvania,15301.0


In [4]:
### Limiting dataframe to relevent data. We will need ID to join to encounter data, Birthdate to calculate age, deathdate to detirmn if paitent died and if so,when
patdf = patdf[['Id','BIRTHDATE','DEATHDATE',]]
## Also, in order to avoid convusion later and to list the patient id as requested, I'll rename ID to PATIENT_ID now
patdf = patdf.rename(columns = {'Id': 'PATIENT_ID'})

In [5]:
### Loading the encounter dataset
encdf = pd.read_csv('encounters.csv')

encdf.head()

,Id,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
0,2590963f-222b-4870-b8c7-86d07f4d44b3,1959-06-18 09:43:49,1959-06-25 21:37:31,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,fa6006c2-c562-3278-859b-96b266fc3ea1,ambulatory,424441002.0,Prenatal initial visit,105.37,72892002.0,Normal pregnancy
1,ba6137e3-c8cd-42e6-9a02-eafa13d1cfa9,2008-08-29 19:14:19,2008-09-07 13:33:07,553b00b2-347c-48ec-90ee-f59924ff50eb,fe803a6f-c717-36e8-a338-9b3cdb9ee724,outpatient,698314001.0,Consultation for treatment,105.37,NaN,NaN
2,10aa7678-5c84-4ce9-81af-09272307c1f0,1999-07-12 03:05:41,1999-07-20 11:18:24,7f4ea9fb-f436-411e-ab34-e94750edfa93,cc6a930c-727f-3999-b655-179e57b30538,wellness,185349003.0,Encounter for check up (procedure),105.37,NaN,NaN
3,3710b40a-bcc0-4c0f-a204-e47a5f3aec98,1997-09-20 16:09:36,1997-09-30 22:43:32,2043c57c-1085-45a3-87e8-a223d5b6693d,d2c4dfee-3c37-318f-a415-367315d28b65,wellness,185349003.0,Encounter for check up (procedure),105.37,NaN,NaN
4,92078752-a89c-4931-b95e-0f8df485e9f6,1963-09-26 09:43:49,1963-10-05 03:20:42,3d8e57b2-3de5-4fb6-95b4-8370270c5cc2,fa6006c2-c562-3278-859b-96b266fc3ea1,ambulatory,424441002.0,Prenatal initial visit,105.37,72892002.0,Normal pregnancy


In [6]:
## Renaming Id to ENCOUNTER_ID

encdf = encdf.rename(columns = {'Id': 'ENCOUNTER_ID'})

###Here I will joing the patients dataframe to the encounter dataframe. 
#The primary key Id in the patent database will join with the Patient Foriegn key in the enconter dataset
df = pd.merge(patdf, encdf, how = "inner", left_on='PATIENT_ID', right_on = 'PATIENT')
## Limiting encounters to after July, 15, 1999. First because Python often reads datetime to as strings, converting back to datetime, mayas well do this for all dates.
df['BIRTHDATE'] = pd.to_datetime(df['BIRTHDATE'])
df['START'] = pd.to_datetime(df['START'])
df['STOP'] = pd.to_datetime(df['STOP'])
df['DEATHDATE'] = pd.to_datetime(df['DEATHDATE'])

In [7]:
df = df.loc[df['START'] > '7/15/1999'] ## limiting to an encounter start date to after 7/15/1999
### The Reason Description has a value of "Drug overdose" which makes limiting the dataset further easy. The data looks already cleaned for typos
df = df.loc[df['REASONDESCRIPTION'] == 'Drug overdose'] 

In [8]:
df.head()

,PATIENT_ID,BIRTHDATE,DEATHDATE,ENCOUNTER_ID,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION
355,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,2a917920-2701-49f0-9340-827320eef76d,2003-03-31 21:50:51,2003-04-08 13:20:43,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
373,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
377,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
381,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,6125f147-72d4-48a4-9813-2a9dc878f055,2014-12-08 21:50:51,2014-12-17 12:25:27,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose
390,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,f837dcf8-af7d-43b8-b5c2-773d494617ab,2015-08-31 21:50:51,2015-09-08 12:04:08,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose


In [9]:
### Now to calculate the date difference in the encounter and the date of birth to get the patient age at the time of encounter.
df['Diff_years'] = df['START'] - df['BIRTHDATE']
### we will now need to calculate it in years. We will use numpy's timedelta function for that
df['Diff_years'] = df.Diff_years / np.timedelta64(1, 'Y')
df = df.loc[df['Diff_years'] < 36] ## age range between 18 and 85
df = df.loc[df['Diff_years'] >= 18]


In [10]:
### I'll need to bring in the medcation dataset in order to count the active medications at the start of the drug overdose encounter
###Fortunatly the medications dataset has a forien key of encounter that joins to ENCOUNTER_ID which joins to ENCOUNTER_ID
meddf = pd.read_csv('medications.csv')
## Because patients are taking multiple medications, I'm going to create a new dataset then count the medications and indicate if there was an opioid,
##then remapp that data back onto my set to avoid duplicates. I only need a few of these collumns so I'll only include those I need

meddf = meddf[['ENCOUNTER','PATIENT','START','STOP','DESCRIPTION']]

#Renaming collumns
meddf = meddf.rename(columns = {'START': 'MED_START','STOP':'MED_STOP','DESCRIPTION':'MED_DESCRIPTION'})

## converting fields to date time, I'll need that to compare that later to the encounter start and end dates
meddf['MED_START'] = pd.to_datetime(meddf['MED_START'])
meddf['MED_STOP'] = pd.to_datetime(meddf['MED_STOP'])


In [11]:
medcountdf = pd.merge(df, meddf, how = "inner", left_on='ENCOUNTER_ID', right_on = 'ENCOUNTER')  ###

In [12]:
medcountdf.head()

,PATIENT_ID,BIRTHDATE,DEATHDATE,ENCOUNTER_ID,START,STOP,PATIENT_x,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION,Diff_years,ENCOUNTER,PATIENT_y,MED_START,MED_STOP,MED_DESCRIPTION


In [13]:
## I don't see any medications for this cohort double checking. There are only two patients admitted with an overdose -b391fad4-d331-4a38-9446-acbd4c300d93
## and b1684d93-7bdc-48d6-a0f9-f73f6bde3ecc who were both over 36 at the time of encounter
## Becauce I could not find any by encounters, I'll try and join Patient to paitent and use the start and end date on Medication to see if the paitent had an active medication 
medcountdf = pd.merge(df, meddf, how = "inner", left_on='PATIENT_ID', right_on = 'PATIENT')



In [14]:
medcountdf.head()  ## Much better, I'll have to count the medication and set the flag. I'll create a function

,PATIENT_ID,BIRTHDATE,DEATHDATE,ENCOUNTER_ID,START,STOP,PATIENT_x,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION,Diff_years,ENCOUNTER,PATIENT_y,MED_START,MED_STOP,MED_DESCRIPTION
0,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,25.311705,a159b9de-5aa6-4d5c-88c5-9b12561cf3b8,708b81c9-21a9-411a-aae4-052ef8925b97,2009-09-10,2010-09-05,Seasonique 91 Day Pack
1,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,25.311705,1e2bb7cc-3d9e-4669-a1ef-03a000c3f4ac,708b81c9-21a9-411a-aae4-052ef8925b97,2011-08-31,2012-08-25,Levora 0.15/30 28 Day Pack
2,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,22874b3d-0873-40e4-a455-897d51d593f7,2012-02-18 21:50:51,2012-02-28 21:12:17,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,25.311705,d391e726-30b5-44db-8aa6-873c1cdc49c8,708b81c9-21a9-411a-aae4-052ef8925b97,2014-08-15,2015-08-14,Mirena 52 MG Intrauterine System
3,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,26.768271,a159b9de-5aa6-4d5c-88c5-9b12561cf3b8,708b81c9-21a9-411a-aae4-052ef8925b97,2009-09-10,2010-09-05,Seasonique 91 Day Pack
4,708b81c9-21a9-411a-aae4-052ef8925b97,1986-10-28,NaT,134c5ee3-1b72-4e3e-a44a-3efebe77603b,2013-08-03 21:50:51,2013-08-13 07:44:52,708b81c9-21a9-411a-aae4-052ef8925b97,fb37c581-84a6-3513-9b1e-1d55e67e887a,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,26.768271,1e2bb7cc-3d9e-4669-a1ef-03a000c3f4ac,708b81c9-21a9-411a-aae4-052ef8925b97,2011-08-31,2012-08-25,Levora 0.15/30 28 Day Pack


<b>Here, I'll count the medication and set the flag. I'll create a function and pass in the dataframe. I’ll create a Counter to count the <u> Active </u> medications at the time of encounter.

Further, I’ll create a dictionary to set the opioid flag.
I can then map the results onto my dataframe
I like using containers like these because they do not expand the dataframe and possible create duplicate values</b>


In [15]:
def medcount(df):
    countdict = Counter() # Instantiating my counter 
    op_list = ['Hydromorphone 325 MG','Fentanyl 100 MCG','Oxycodone-acetaminophen 100ML'] ## our list of opioids
    op_flag_dict = {} # the dictonary
    
    for i, row in df.iterrows(): ## looping through the dataframe
        if row['START'] >= row['MED_START']:
            if (row['STOP'] <= row['MED_STOP'] or pd.isnull(row['MED_STOP'])):  # is the medication active at the time of encounter
                
                countdict[row['ENCOUNTER_ID']] += 1  # Ticking up one for each into the counter
                
                
                if row['MED_DESCRIPTION'] in op_list: # is the medicaion one of the three opioids
                                       
                    op_flag_dict[row['ENCOUNTER_ID']] =  '1' #The encounter ID is the key and the #1 is the value in the dictonary

                
    return(countdict,op_flag_dict)  #returning counter and dictonary

In [16]:
medcount_dict, opioid_flag_dict = medcount(medcountdf) # running function

In [17]:
## Mapping values
df['COUNT_CURRENT_MEDS'] = df['ENCOUNTER_ID'].map(medcount_dict)
df['CURRENT_OPIOID_IND'] = df['ENCOUNTER_ID'].map(opioid_flag_dict)

In [18]:
### sorting values to list paitient then time of admittance in order to get difference in time, calclate if death date is in encounter
df = df.sort_values(['PATIENT_ID','START'], ascending=[True, False])

<b>Here,  I’ll create another function that iterates over the dataframe. I can accomplish a lot of the tasks.

1)	I’m going to calculate if the patients death occurred during the encounter, or at a later time and fill accordingly<br>

2)	As I loop through the dataframe I’m going to check is the patient was seen before via a set, and if so starting filling a list for last remission date. I’ll be able to access this date from the list later in the function. I’ll also calculate the time difference on this patients subsequent visit to the last remission date <br>
3)	Using my list of dates and time difference calculation I’ll check if it’s within 90 and 30 days


In [19]:
def additional(df):
    seen = set()  ## creating set to track if paitent returns
    readmission_seen = set() # to track if paitent had a remmision in 90 days
    remission_list = [] ## to fill the line above  Start date you can access that via the 2nd to last index [-2]
    remissiondate_list = [] # to fill the firsy readmission date, it will only fill the first time a paitent is within 90 days thanks to the readmission_seen
    for i, row in df.iterrows(): #looping over dataframe
       
        if (row['DEATHDATE'] >= row['START'] and row['DEATHDATE'] <= row['STOP']): 
           
            # to see if the paitent died during the vsist
            df.loc[i,'DEATH_AT_VISIT_IND'] = '1'
        elif pd.isnull(row['DEATHDATE']):
            df.loc[i,'DEATH_AT_VISIT_IND'] = ''
                        
        else:
            df.loc[i,'DEATH_AT_VISIT_IND'] = '0'
               
        remission_list.append(row['START']) ## filling the list to access the row aboves date
        
        # is this a new paitent
        if row['PATIENT_ID'] in seen: # 
            last_readmit = remission_list[-2] # the row aboves start date
            time_difference = last_readmit - row['START'] # calculating time difference
        
            time_difference = time_difference / np.timedelta64(1, 'D')
          
            ## Within 90 days
            if time_difference <= 90:
                df.loc[i,'READMISSION_90_DAY_IND'] = 1
          
            else:
                df.loc[i,'READMISSION_90_DAY_IND'] = 0
                
            ##within 30
            if time_difference <= 30:
                df.loc[i,'READMISSION_30_DAY_IND'] = 1
            else:
                df.loc[i,'READMISSION_30_DAY_IND'] = 0
                
        else:
            ## adding patient to set, filling defualt values of 0
            seen.add(row['PATIENT_ID'])
            df.loc[i,'READMISSION_90_DAY_IND'] = 0
            df.loc[i,'READMISSION_30_DAY_IND'] = 0
        
    return(df)
            
            
        
        

In [20]:
df1 = additional(df)

In [21]:
df1.head() #exploring

,PATIENT_ID,BIRTHDATE,DEATHDATE,ENCOUNTER_ID,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION,Diff_years,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND,DEATH_AT_VISIT_IND,READMISSION_90_DAY_IND,READMISSION_30_DAY_IND
33121,038eb759-c1e9-47f7-a131-afe61950d65e,1996-12-16,NaT,30a506a6-916d-4862-b835-ba755b62d7b1,2018-01-06 14:26:11,2018-01-15 21:27:39,038eb759-c1e9-47f7-a131-afe61950d65e,e85acdd2-1c4c-3350-b7dd-033126c3a616,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,21.058890,1,NaN,,0.0,0.0
171651,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,c706c531-230b-41ad-8d7f-93f8a958540e,2013-09-11 20:07:38,2013-09-17 19:09:05,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,33.103592,1,NaN,1,0.0,0.0
171650,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,b8f11c48-f217-4ff5-9047-4952af6f305f,2012-07-11 20:07:38,2012-07-20 14:40:09,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,31.934506,1,NaN,0,0.0,0.0
171628,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,5fe7c493-9eb6-4663-9a2f-2d795277be83,2009-07-15 20:07:38,2009-07-25 04:32:45,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,28.944711,1,NaN,0,0.0,0.0
171627,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,0ec793ed-8f41-4cf8-a759-0b5283f504a6,2006-02-11 20:07:38,2006-02-21 23:17:25,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,25.522327,1,NaN,0,0.0,0.0


In [22]:
df1 = df1.sort_values(['PATIENT_ID','START'], ascending=[True,True])
Admission_dict = {}

<b> In order to get the first remission date, I’ll need the dates to reverse<br>
Much like the last function, I’ll need to check if the difference is more than 90 days.

I’ll create two sets: One to see if the patient was seen, then calculate the dates as with the last function, and the second set to see if the patient already had a 90 day encounter before in order to make sure I’m using the first date.

This was the most challenging part of the exercise, but a lot of fun to figure out!


In [23]:
def firstdate(df):
    
    seen = set()  ## creating set to track if paitent returns
    readmission_seen = set() # to track if paitent had a remmision in 90 days
    remission_list = [] ## to fill the line above  Start date you can access that via the 2nd to last index [-2]
    remissiondate_list = [] # to fill the firsy readmission date, it will only fill the first time a paitent is within 90 days thanks to the readmission_seen
    patrem_seen = set()
    
    for i, row in df.iterrows(): #looping over dataframe
        
        
        remission_list.append(row['START']) ## filling the list to access the row aboves date
        
        # is this a new paitent
        if row['PATIENT_ID'] in seen: # 
            last_readmit = remission_list[-2] # the row aboves start date
            time_difference = last_readmit - row['START'] # calculating time difference
         
            time_difference = time_difference / np.timedelta64(1, 'D')
          
            ## Within 90 days
            if time_difference >= -90:
                
                # Does the paitent have a 90 day encounter?
                if row['PATIENT_ID'] not in readmission_seen:
                    first_readmission_date = last_readmit
                    
                    
                    readmission_seen.add(row['PATIENT_ID'])
                    
                    Admission_dict[row['PATIENT_ID']] =  row['START']
                    
                    
                    
                    remissiondate_list.append(row['START'])
                else:
                    if row['PATIENT_ID'] not in patrem_seen:
                        
                        Admission_dict[row['PATIENT_ID']] =  remissiondate_list[-1]
                        
         
        else:
            seen.add(row['PATIENT_ID'])

        
    return(df)
            
            
        

In [24]:
firstdate(df1)

,PATIENT_ID,BIRTHDATE,DEATHDATE,ENCOUNTER_ID,START,STOP,PATIENT,PROVIDER,ENCOUNTERCLASS,CODE,DESCRIPTION,COST,REASONCODE,REASONDESCRIPTION,Diff_years,COUNT_CURRENT_MEDS,CURRENT_OPIOID_IND,DEATH_AT_VISIT_IND,READMISSION_90_DAY_IND,READMISSION_30_DAY_IND
33121,038eb759-c1e9-47f7-a131-afe61950d65e,1996-12-16,NaT,30a506a6-916d-4862-b835-ba755b62d7b1,2018-01-06 14:26:11,2018-01-15 21:27:39,038eb759-c1e9-47f7-a131-afe61950d65e,e85acdd2-1c4c-3350-b7dd-033126c3a616,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,21.058890,1,NaN,,0.0,0.0
171627,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,0ec793ed-8f41-4cf8-a759-0b5283f504a6,2006-02-11 20:07:38,2006-02-21 23:17:25,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,25.522327,1,NaN,0,0.0,0.0
171628,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,5fe7c493-9eb6-4663-9a2f-2d795277be83,2009-07-15 20:07:38,2009-07-25 04:32:45,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,28.944711,1,NaN,0,0.0,0.0
171650,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,b8f11c48-f217-4ff5-9047-4952af6f305f,2012-07-11 20:07:38,2012-07-20 14:40:09,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,31.934506,1,NaN,0,0.0,0.0
171651,04aefd77-78d8-4590-a7fb-92b9a49df0f1,1980-08-05,2013-09-17,c706c531-230b-41ad-8d7f-93f8a958540e,2013-09-11 20:07:38,2013-09-17 19:09:05,04aefd77-78d8-4590-a7fb-92b9a49df0f1,d124f9a7-be71-3934-b71f-56f932fc0d38,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,33.103592,1,NaN,1,0.0,0.0
119486,072f5b39-4272-4b4f-8292-f3042fad3d7d,1988-06-08,NaT,7dae5670-a841-4d1e-a4b2-466cb0374be9,2006-07-04 17:53:08,2006-07-12 10:03:22,072f5b39-4272-4b4f-8292-f3042fad3d7d,fc63d100-1b79-3246-bb97-d770c4bcf7e4,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,18.072227,0,NaN,,0.0,0.0
119487,072f5b39-4272-4b4f-8292-f3042fad3d7d,1988-06-08,NaT,d81cc54f-9377-4728-9346-3a53cc89594a,2009-09-10 17:53:08,2009-09-18 05:20:06,072f5b39-4272-4b4f-8292-f3042fad3d7d,fc63d100-1b79-3246-bb97-d770c4bcf7e4,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,21.259150,0,NaN,,0.0,0.0
119490,072f5b39-4272-4b4f-8292-f3042fad3d7d,1988-06-08,NaT,fcac5f16-3824-4a15-9d12-3c915d3f5846,2010-01-09 17:53:08,2010-01-18 22:43:40,072f5b39-4272-4b4f-8292-f3042fad3d7d,fc63d100-1b79-3246-bb97-d770c4bcf7e4,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,21.590437,0,NaN,,0.0,0.0
119546,072f5b39-4272-4b4f-8292-f3042fad3d7d,1988-06-08,NaT,e88d0d49-8c44-4b10-9b39-7bcef3b3996f,2016-03-06 17:53:08,2016-03-14 11:59:48,072f5b39-4272-4b4f-8292-f3042fad3d7d,fc63d100-1b79-3246-bb97-d770c4bcf7e4,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,27.745252,1,NaN,,0.0,0.0
223451,0805e07d-3889-4027-bd93-0e848ca14baf,1976-06-25,NaT,2e265935-b642-4178-b807-0dbb4fef36b1,2011-08-10 00:05:39,2011-08-15 21:42:01,0805e07d-3889-4027-bd93-0e848ca14baf,f21e913e-bf44-3fe8-8d85-fa67d7f6e04b,emergency,50849002.0,Emergency Room Admission,105.37,55680006.0,Drug overdose,35.124620,0,NaN,,0.0,0.0


In [25]:
## Mapping the dictonary on
df1['FIRST_READMISSION_DATE'] = df1['PATIENT_ID'].map(Admission_dict)

In [26]:
#Limiting the names
df1 = df1[['PATIENT_ID','ENCOUNTER_ID','START','Diff_years','DEATH_AT_VISIT_IND','COUNT_CURRENT_MEDS','CURRENT_OPIOID_IND','READMISSION_90_DAY_IND','READMISSION_30_DAY_IND','FIRST_READMISSION_DATE']]

In [27]:
#Changing the names, as requested
df1 = df1.rename(columns = {'START': 'HOSPITAL_ENCOUNTER_DATE','Diff_years':'AGE_AT_VISIT'})

In [28]:
##### Exporting to Excel spreadsheet
from pandas import ExcelWriter

writer = ExcelWriter('Joey.Logan.xlsx')
df1.to_excel(writer,'Joey.Logan')

writer.save()